# LangGraph: Planning and Execution with Graphs

This notebook illustrates the concept of langgraph: planning and execution with graphs using LLMs.

# LangGraph: Planning and Execution with Graphs

In this notebook, we build a basic LangGraph to simulate a Plan-and-Act agent. The LLM generates subtasks (planning), and the executor executes each subtask in order.

In [1]:
# Install LangGraph and LangChain (uncomment if necessary)
# !pip install langgraph langchain openai


In [2]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

ModuleNotFoundError: No module named 'langgraph'

## Step 1: Define the nodes
- The `planner` node will turn the user goal into subtasks
- The `executor` node will execute subtasks sequentially

In [ ]:
# Simple in-memory state to track progress
state = {"goal": "Find the capital of France and calculate the length of its name"}

In [ ]:
# Node: Planner
def planner_node(state):
    return {
        "steps": [
            "Find the capital of France",
            "Calculate the number of letters in the capital's name"
        ],
        "step_index": 0
    }

In [ ]:
# Node: Executor
def executor_node(state):
    step = state['steps'][state['step_index']]
    response = llm.predict(step)
    print(f"Executing: {step}\n→ {response}\n")
    return {
        **state,
        "step_index": state['step_index'] + 1
    }

## Step 2: Build and compile the LangGraph

In [ ]:
# Build the graph
workflow = StateGraph()
workflow.add_node("planner", planner_node)
workflow.add_node("executor", executor_node)

# Define flow: planner → executor → END
workflow.set_entry_point("planner")
workflow.add_edge("planner", "executor")
workflow.add_edge("executor", END)

app = workflow.compile()

## Step 3: Run the graph

In [ ]:
# Run the workflow
result = app.invoke(state)
result